In [16]:
from __future__ import print_function

from rdkit import Chem
from rdkit.Chem import AllChem

In [28]:
import random, copy

def cano(smiles): # canonicalize smiles by MolToSmiles function
    return Chem.MolToSmiles(Chem.MolFromSmiles(smiles))

def block(ch, smiles):
    return (ch + cano(smiles)) if (smiles != '') else cano(smiles)

# choose carboxylic acid (tmp)
acid_smi = 'CC(C)CCC(O)=O'
acid = Chem.MolFromSmiles(acid_smi)

# choose alcohol (tmp)
alcohol_smi = 'CC(C)CO'

# choose amine (tmp)
amine_smi = 'CC(C)CN'

In [29]:
## Carboxylic acid

DIAZO = AllChem.ReactionFromSmarts('[C:1](=O)[O:2].[C:3]=[N+]=[N-]>>[C:1](=O)[O:2][C:3]')
RED = AllChem.ReactionFromSmarts('[C:1](=O)[O:2]>>[C:1][O:2]')
RLI = AllChem.ReactionFromSmarts('[C:1](=[O:2])O.[C:3][Li]>>[C:1](=[O:2])[C:3]')
CHLORI = AllChem.ReactionFromSmarts('[C:1](=[O:2])O>>[C:1](=[O:2])Cl')
HVZ = AllChem.ReactionFromSmarts('[CH2:1][C:2](=O)O>>[C:1](Br)[C:2](=O)O')
ESTER = AllChem.ReactionFromSmarts('[C:1](=[O:2])[O:3].[O:4]>>[C:1](=[O:2])[O:4]')

acid_reactions_list = [
    
    ('DIAZO', DIAZO, ['C=[N+]=[N-]'], ['']),
    ('RED', RED, [''], ['[H-].[H-].[H-].[H-].[Al+3].[Li+]', 'B.C1CCOC1']),
    ('RLI', RLI, ['[Li]C', '[Li]CC', '[Li]CCC', '[Li]CCCC', '[Li]c1ccccc1'], ['']),
    ('CHLORI', CHLORI, [''], ['ClS(Cl)=O']),
    ('HVZ', HVZ, [''], ['BrBr.BrP(Br)Br']),
    ('ESTER', ESTER, [''], ['Cl', 'OS(O)(=O)=O']),
    
]

# general reactions
for reaction in acid_reactions_list:
    print(reaction[0])
    for reagent in reaction[2]:
        if reaction[0] == 'ESTER':
            reagent = alcohol_smi
        if reagent == '': products = reaction[1].RunReactants((acid,))
        else: products = reaction[1].RunReactants((acid, Chem.MolFromSmiles(reagent)))
        if len(products) == 0: continue
        product_smi = [Chem.MolToSmiles(product) for product in products[0]]
        for sub_reagent in reaction[3]:
            print(cano(acid_smi) + block('.', reagent) + '>' + cano(sub_reagent) + '>' + '.'.join(product_smi))

DIAZO
CC(C)CCC(=O)O.C=[N+]=[N-]>>COC(=O)CCC(C)C
RED
CC(C)CCC(=O)O>[Al+3].[H-].[H-].[H-].[H-].[Li+]>CC(C)CCCO
CC(C)CCC(=O)O>B.C1CCOC1>CC(C)CCCO
RLI
CC(C)CCC(=O)O.[Li]C>>CC(=O)CCC(C)C
CC(C)CCC(=O)O.[Li]CC>>CCC(=O)CCC(C)C
CC(C)CCC(=O)O.[Li]CCC>>CCCC(=O)CCC(C)C
CC(C)CCC(=O)O.[Li]CCCC>>CCCCC(=O)CCC(C)C
CHLORI
CC(C)CCC(=O)O>O=S(Cl)Cl>CC(C)CCC(=O)Cl
HVZ
CC(C)CCC(=O)O>BrBr.BrP(Br)Br>CC(C)CC(Br)C(=O)O
ESTER
CC(C)CCC(=O)O.CC(C)CO>Cl>CC(C)CCC(=O)OCC(C)C
CC(C)CCC(=O)O.CC(C)CO>O=S(=O)(O)O>CC(C)CCC(=O)OCC(C)C


In [31]:
chloride = CHLORI.RunReactants((acid,))[0][0]
chloride_smi = Chem.MolToSmiles(chloride)

In [32]:
ANHYD = AllChem.ReactionFromSmarts('[C:1](=O)[Cl:2].[C:3](=O)[O:4]>>[C:1](=O)[O:4][C:3](=O)')
ESTER = AllChem.ReactionFromSmarts('[C:1](=O)[Cl:2].[O:3]>>[C:1](=O)[O:3]')
AMIDE = AllChem.ReactionFromSmarts('[C:1](=O)[Cl:2].[N:3]>>[C:1](=O)[N:3]')

chloride_reactions_list = [
    
    ('ANHYD', ANHYD, [''], ['']),
    ('ESTER', ESTER, [''], ['']),
    ('AMIDE', AMIDE, [''], ['']),
    
]

acid = chloride

# general reactions
for reaction in chloride_reactions_list:
    print(reaction[0])
    for reagent in reaction[2]:
        if reaction[0] == 'ANHYD':
            reagent = acid_smi
        elif reaction[0] == 'ESTER':
            reagent = alcohol_smi
        elif reaction[0] == 'AMIDE':
            reagent = amine_smi
        if reagent == '': products = reaction[1].RunReactants((acid,))
        else: products = reaction[1].RunReactants((acid, Chem.MolFromSmiles(reagent)))
        if len(products) == 0: continue
        product_smi = [Chem.MolToSmiles(product) for product in products[0]]
        for sub_reagent in reaction[3]:
            print(cano(chloride_smi) + block('.', reagent) + '>' + cano(sub_reagent) + '>' + '.'.join(product_smi))

ANHYD
CC(C)CCC(=O)Cl.CC(C)CCC(=O)O>>CC(C)CCC(=O)OC(=O)CCC(C)C
ESTER
CC(C)CCC(=O)Cl.CC(C)CO>>CC(C)CCC(=O)OCC(C)C
AMIDE
CC(C)CCC(=O)Cl.CC(C)CN>>CC(C)CCC(=O)NCC(C)C


In [ ]:
ESTER
AMIDE
CARBO

anhydride = ANHYD.RunReactants((acid, Chem.MolFromSmiles(acid_smi)))
